In [ ]:
import os
import json
import numpy as np
import nibabel as nib
from nilearn.image import resample_to_img
from skimage import measure
import trimesh

# ==========================
# CONFIG
# ==========================
ATLAS_PATH = "BN_Atlas_246_1mm.nii.gz"
OUTPUT_DIR = "region_meshes"
METADATA_PATH = "region_metadata.json"

VOXEL_THRESHOLD = 0.5   # for probabilistic atlases
MIN_REGION_SIZE = 500   # voxels (skip tiny junk regions)

os.makedirs(OUTPUT_DIR, exist_ok=True)

# ==========================
# LOAD ATLAS
# ==========================
atlas_img = nib.load(ATLAS_PATH)
atlas_data = atlas_img.get_fdata()
affine = atlas_img.affine

# Region IDs (exclude background 0)
region_ids = sorted([int(i) for i in np.unique(atlas_data) if i > 0])

metadata = {}

print(f"Found {len(region_ids)} regions")



Found 246 regions
Processing region 1


AttributeError: module 'skimage.measure' has no attribute 'marching_cubes'

In [ ]:
# ==========================
# PROCESS EACH REGION
# ==========================
for region_id in region_ids:
    print(f"Processing region {region_id}")
    mask = atlas_data == region_id

    if np.sum(mask) < MIN_REGION_SIZE:
        print("  Skipped (too small)")
        continue
    verts, faces, normals, _ = measure.marching_cubes_lewiner(
        mask.astype(np.uint8),
        level=0.5
    )

    # Convert voxel coords → world coords (MNI)
    verts_h = np.c_[verts, np.ones(len(verts))]
    verts_world = (affine @ verts_h.T).T[:, :3]

    # Create mesh
    mesh = trimesh.Trimesh(
        vertices=verts_world,
        faces=faces,
        vertex_normals=normals,
        process=False
    )

    # Export GLB
    out_path = os.path.join(OUTPUT_DIR, f"region_{region_id}.glb")
    mesh.export(out_path)

    # Save metadata
    metadata[str(region_id)] = {
        "region_id": int(region_id),
        "mesh": f"region_{region_id}.glb",
        "n_vertices": int(len(verts)),
        "n_faces": int(len(faces))
    }


with open(METADATA_PATH, "w") as f:
    json.dump(metadata, f, indent=2)



Processing region 1
Processing region 2
Processing region 3
Processing region 4
Processing region 5
Processing region 6
Processing region 7
Processing region 8
Processing region 9
Processing region 10
Processing region 11
Processing region 12
Processing region 13
Processing region 14
Processing region 15
Processing region 16
Processing region 17
Processing region 18
Processing region 19
Processing region 20
Processing region 21
Processing region 22
Processing region 23
Processing region 24
Processing region 25
Processing region 26
Processing region 27
Processing region 28
Processing region 29
Processing region 30
Processing region 31
Processing region 32
Processing region 33
Processing region 34
Processing region 35
Processing region 36
Processing region 37
Processing region 38
Processing region 39
Processing region 40
Processing region 41
Processing region 42
Processing region 43
Processing region 44
Processing region 45
Processing region 46
Processing region 47
Processing region 48
P

In [ ]:
import nibabel as nib
import numpy as np
import json
from nilearn.image import resample_to_img

# Load images
fmri_img = nib.load("sub-01_ses-test_task-covertverbgeneration_bold.nii.gz")
atlas_img = nib.load("BN_Atlas_246_1mm.nii.gz")

atlas_resampled = resample_to_img(
    atlas_img,
    fmri_img,
    interpolation="nearest")

fmri_data = fmri_img.get_fdata()
atlas_data = atlas_resampled.get_fdata().astype(int)

print("fMRI shape:", fmri_data.shape)
print("Resampled atlas shape:", atlas_data.shape)

fMRI shape: (64, 64, 30, 173)
Resampled atlas shape: (64, 64, 30)


In [ ]:
import json

with open("timeseries.json", "w") as f:
    json.dump(region_timeseries, f)


In [ ]:
import numpy as np
import json

region_timeseries = {}

region_ids = np.unique(atlas_data)
region_ids = region_ids[region_ids != 0]

for rid in region_ids:
    mask = atlas_data == rid
    # Extract voxel time series and average
    ts = fmri_data[mask].mean(axis=0)
    region_timeseries[int(rid)] = ts.tolist()

with open("timeseries.json", "w") as f:
    json.dump(region_timeseries, f)


In [ ]:
import trimesh
import trimesh.exchange.gltf as gltf

scene = trimesh.Scene()

for i in range(1, 247):
    mesh = trimesh.load(f'./region_meshes/region_{i}.glb')
    scene.add_geometry(mesh, node_name=f'region_{i}')

scene.export('all_regions.glb')


In [ ]:
from nilearn import datasets
atlas = datasets.fetch_atlas_brainnetome()
print(atlas.labels)  

C:\Users\User\AppData\Roaming\Python\Python37\site-packages\nilearn\__init__.py:67: FutureWarning: Python 3.7 support is deprecated and will be removed in release 0.12 of Nilearn. Consider switching to Python 3.9 or 3.10.
  _python_deprecation_warnings()


AttributeError: module 'nilearn.datasets' has no attribute 'fetch_atlas_brainnetome'